In [1]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score


In [2]:
data = pd.read_csv("data.csv")
data.head()


,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,...,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v,pose,correct,step
0,0.471043,0.715024,-0.318419,0.981122,0.425809,0.652623,-0.194296,0.995248,0.543037,0.668582,...,0.361520,0.435238,0.339473,0.531366,0.329219,0.451995,0.185829,0,0,0
1,0.458709,0.762975,-0.171610,0.996881,0.487831,0.674606,-0.138267,0.996137,0.475767,0.644857,...,0.395488,0.474359,0.080846,0.514495,0.397352,0.467766,0.055228,0,0,0
2,0.461469,0.731856,-0.231723,0.941787,0.449222,0.629312,-0.139082,0.988798,0.531250,0.650280,...,0.454985,0.419889,0.085213,0.574922,0.453711,0.433322,0.084213,0,0,0
3,0.667741,0.400317,0.442511,0.729667,0.641468,0.435358,0.370686,0.999631,0.685328,0.397876,...,0.714341,-0.585603,0.952009,0.398631,0.726997,-0.651245,0.788665,0,0,0
4,0.438507,0.602888,-0.294648,0.997777,0.491524,0.583383,-0.214972,0.999542,0.485241,0.670385,...,0.671250,0.368280,0.676167,0.524678,0.682319,0.425517,0.250376,0,0,0


In [3]:
features = data.drop(["pose", "correct", "step"], axis=1)
target = data[["pose", "correct", "step"]]

Xtrain, xtest, Ytrain, ytest = train_test_split(
    features, target, test_size=0.2, random_state=1234)


In [4]:
model = RandomForestClassifier()
classifier = MultiOutputClassifier(model, n_jobs=-1)
classifier.fit(Xtrain, Ytrain)


MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1)

In [5]:
print(classifier.score(xtest, np.array(ytest)))


0.50472334682861


In [21]:
import mediapipe as mp
import cv2
import os


In [22]:
mp_pose = mp.solutions.pose


In [23]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")


In [24]:
full_list = []

main_dir = "./DATASET/TEST"
pose_dir_list = os.listdir(main_dir)

for j in range(4, len(pose_dir_list)):
    dir_list = os.listdir(f"{main_dir}/{pose_dir_list[j]}")

    for i in range(108, len(dir_list)):

        pre_list = []

        with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
            image = cv2.imread(f"{main_dir}/{pose_dir_list[j]}/{dir_list[i]}")
            result = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            try:
                predict = True
                for landmarks in result.pose_landmarks.landmark:
                    pre_list.append(landmarks)
            except AttributeError:
                print(
                    f"No points on {main_dir}/{pose_dir_list[j]}/{dir_list[i]}")
                predict = False

        if predict == True:
            gen1116 = np.array([
                [
                    pre_list[k].x,
                    pre_list[k].y,
                    pre_list[k].z,
                    pre_list[k].visibility
                ] for k in range(11, 17)
            ]).flatten().tolist()

            gen2333 = np.array([
                [
                    pre_list[k].x,
                    pre_list[k].y,
                    pre_list[k].z,
                    pre_list[k].visibility
                ] for k in range(23, 33)
            ]).flatten().tolist()

            gen1116.extend(gen2333)

            all_list = [
                pre_list[0].x,
                pre_list[0].y,
                pre_list[0].z,
                pre_list[0].visibility,
            ]

            all_list.extend(gen1116)


In [25]:
data_x = pd.DataFrame([all_list], columns=col_names)


In [26]:
pred = model.predict(data_x)


In [27]:
pose_to_class_name = {
    0: "downdog",
    1: "goddess",
    2: "plank",
    3: "tree",
    4: "warrior2"
}


In [28]:
print(f"The yoga pose is: {pose_to_class_name[pred[0]]}")


The yoga pose is: warrior2


In [29]:
pk.dump(model, open("./models/rfc.model", "wb"))


In [30]:
loaded_model = pk.load(open("./models/rfc.model", "rb"))
result = loaded_model.score(xtest, ytest)
print(result)


0.9425837320574163
